In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from torch import nn
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['covertype_forest.csv']


In [2]:
import torch
def grad_example(x,w): #variant 3 
    w.requires_grad_(True)
    ex = torch.exp(-x.matmul(w))
    res = 1/(1+ex)
    res.backward()
    print(w.grad)
x = torch.tensor([1.,2.,3.])
w = torch.tensor([2.,3.,4.])
grad_example(x,w)

tensor([2.0612e-09, 4.1223e-09, 6.1835e-09])


In [3]:
cover_df = pd.read_csv("../input/covertype_forest.csv")

In [4]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder().fit(cover_df["class"])
cover_target = label_encoder.transform(cover_df["class"])
print(cover_target)

[4 6 6 ... 1 1 1]


In [5]:
from sklearn.model_selection import train_test_split
cover_df = cover_df.drop("class",axis=1)
df_train, df_test, y_train, y_test = train_test_split(cover_df, cover_target, test_size=0.15, stratify=cover_target)

In [6]:
to_normalize = [(i, col) for i, col in enumerate(cover_df.columns)
                        if not col.startswith('wilderness_area') and not col.startswith('soil_type')]
idx_to_normalize = [i for i,col in to_normalize] #номера столбцов
columns_to_normalize = [col for i, col in to_normalize] #названия

print(columns_to_normalize)
print(idx_to_normalize)
from torch.utils.data import TensorDataset,DataLoader
tensor_train = torch.from_numpy(df_train.values).type(torch.FloatTensor)
tensor_test = torch.from_numpy(df_test.values).type(torch.FloatTensor)
train_mean = torch.mean(tensor_train[:,idx_to_normalize], dim=0)
train_std = torch.std(tensor_train[:,idx_to_normalize], dim=0)

tensor_train[:,idx_to_normalize] -= train_mean
tensor_train[:,idx_to_normalize] /= train_std
tensor_test[:,idx_to_normalize] -= train_mean
tensor_test[:,idx_to_normalize] /= train_std

['elevation', 'aspect', 'slope', 'horizontal_distance_to_hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [7]:
y_train_tensor = torch.from_numpy(y_train).type(torch.LongTensor)
y_test_tensor = torch.from_numpy(y_test).type(torch.LongTensor)

train_ds = TensorDataset(tensor_train, y_train_tensor)
test_ds = TensorDataset(tensor_test, y_test_tensor)
train_loader = DataLoader(train_ds,batch_size=256, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=256)

In [8]:
for xx, yy in train_loader:
    print(xx)
    print(yy)
    break

tensor([[-0.5311,  1.5348,  0.2564,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.6428,  0.3364, -0.8154,  ...,  0.0000,  0.0000,  0.0000],
        [-1.2841,  1.5616,  1.1942,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 1.1317,  1.7494,  0.6583,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.6812, -0.2181, -0.0116,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.6214, -1.1661, -0.8154,  ...,  0.0000,  0.0000,  0.0000]])
tensor([4, 4, 4, 6, 2, 4, 5, 4, 6, 4, 2, 4, 5, 6, 4, 6, 6, 4, 6, 4, 6, 4, 6, 4,
        4, 4, 4, 4, 6, 2, 6, 4, 4, 4, 6, 4, 6, 4, 6, 3, 4, 4, 6, 5, 4, 2, 4, 4,
        4, 4, 4, 6, 5, 4, 6, 4, 4, 6, 6, 4, 6, 0, 4, 4, 4, 1, 4, 6, 1, 4, 4, 4,
        4, 6, 4, 4, 3, 6, 5, 4, 4, 4, 6, 4, 4, 6, 6, 4, 4, 4, 6, 4, 2, 6, 6, 4,
        6, 4, 4, 6, 6, 4, 4, 2, 0, 4, 5, 4, 4, 6, 4, 3, 4, 4, 6, 5, 6, 4, 4, 3,
        4, 5, 6, 4, 4, 4, 4, 6, 4, 4, 4, 6, 4, 4, 4, 4, 6, 0, 4, 4, 5, 6, 6, 4,
        6, 4, 4, 3, 4, 6, 6, 6, 4, 6, 4, 4, 3, 6, 4, 0, 2, 6, 6, 6, 4, 6, 6, 4,
      

In [9]:
class my_net:
    def __init__(self):
        self.w_inp = torch.randn(54,200).type(torch.FloatTensor).requires_grad_(True)
        self.b_inp = torch.randn(200).type(torch.FloatTensor).requires_grad_(True)
        self.w_out = torch.randn(200,7).type(torch.FloatTensor).requires_grad_(True)
        self.b_out = torch.randn(7).type(torch.FloatTensor).requires_grad_(True)
    def train(self, loader,lr,epoch):
        for i in range(epoch):
            for xx,yy in loader:
                if self.w_inp.grad is not None:
                    self.w_inp.grad.zero_()
                    self.b_inp.grad.zero_()
                    self.w_out.grad.zero_()
                    self.b_out.grad.zero_()
                h = xx.matmul(self.w_inp) + self.b_inp
                h = h.relu()
                out = h.matmul(self.w_out) + self.b_out
                out = out.log_softmax(dim = 1)
                loss = -(1/len(xx))*out[torch.arange(len(xx)),yy].sum()
                loss.backward()
                with torch.no_grad():
                    self.w_inp -= self.w_inp.grad*lr
                    self.b_inp -= self.b_inp.grad*lr
                    self.w_out -= self.w_out.grad*lr
                    self.b_out -= self.b_out.grad*lr
            print(i,"loss: ", loss)
    def pred(self,loader):
        res = []
        for xx,yy in loader:
                h = xx.matmul(self.w_inp) + self.b_inp
                h = h.relu()
                out = h.matmul(self.w_out) + self.b_out
                out = out.log_softmax(dim = 1)
                res.append(out.argmax(dim=1).numpy())
        return res
    def predict(self,xx):
        h = xx.matmul(self.w_inp) + self.b_inp
        h = h.relu()
        out = h.matmul(self.w_out) + self.b_out
        out = out.log_softmax(dim = 1)
        return out

In [10]:
net = my_net()

In [11]:
net.train(train_loader,0.5,20)

0 loss:  tensor(1.1926, grad_fn=<MulBackward0>)
1 loss:  tensor(1.2346, grad_fn=<MulBackward0>)
2 loss:  tensor(0.8325, grad_fn=<MulBackward0>)
3 loss:  tensor(0.8479, grad_fn=<MulBackward0>)
4 loss:  tensor(0.7526, grad_fn=<MulBackward0>)
5 loss:  tensor(0.6585, grad_fn=<MulBackward0>)
6 loss:  tensor(0.8211, grad_fn=<MulBackward0>)
7 loss:  tensor(0.7453, grad_fn=<MulBackward0>)
8 loss:  tensor(0.8047, grad_fn=<MulBackward0>)
9 loss:  tensor(0.8232, grad_fn=<MulBackward0>)
10 loss:  tensor(0.7527, grad_fn=<MulBackward0>)
11 loss:  tensor(0.7177, grad_fn=<MulBackward0>)
12 loss:  tensor(0.6192, grad_fn=<MulBackward0>)
13 loss:  tensor(0.6642, grad_fn=<MulBackward0>)
14 loss:  tensor(0.6661, grad_fn=<MulBackward0>)
15 loss:  tensor(0.8088, grad_fn=<MulBackward0>)
16 loss:  tensor(0.6200, grad_fn=<MulBackward0>)
17 loss:  tensor(0.6211, grad_fn=<MulBackward0>)
18 loss:  tensor(0.5974, grad_fn=<MulBackward0>)
19 loss:  tensor(0.7813, grad_fn=<MulBackward0>)


In [12]:
from sklearn.metrics import classification_report
y_true = []
y_pred = []
for xx,yy in test_loader:
    out = net.predict(xx)
    for i in out:
        y_pred.append(int(i.argmax()))
    for i in yy:
        y_true.append(int(i))
print(classification_report(y_pred,y_true))

              precision    recall  f1-score   support

           0       0.21      0.58      0.31       144
           1       0.10      0.72      0.18        29
           2       0.28      0.61      0.39       279
           3       0.61      0.81      0.70       509
           4       0.85      0.78      0.81      8392
           5       0.79      0.68      0.73      1294
           6       0.78      0.77      0.77      5912

   micro avg       0.77      0.77      0.77     16559
   macro avg       0.52      0.71      0.56     16559
weighted avg       0.80      0.77      0.78     16559

